follow from https://qiita.com/koshian2/items/f4917b1e6c944674fb80

In [ ]:
# lib import
import keras
from keras.datasets   import mnist
from keras.models     import Model
from keras.layers     import Dense, Dropout, BatchNormalization, Input, Activation, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import RMSprop

import numpy          as     np


In [ ]:
# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()


In [ ]:
X_train  = X_train.astype('float32')
X_test   = X_test.astype('float32')
X_train /= 255.0
X_test  /= 255.0
X_train  = X_train[:, :, :, np.newaxis]
X_test   = X_test[:, :, :, np.newaxis]


print('X_train.shape = (%d, %d, %d, %d)' % X_train.shape)
print('X_test.shape  = (%d, %d, %d, %d)' % X_test.shape)


In [ ]:
num_class = np.max(y_train) + 1

print('y_train.shape = %d' % y_train.shape)
print('y_test.shape  = %d' % y_test.shape)
print('num_class     = %d' % num_class)

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_class)
y_test  = keras.utils.to_categorical(y_test,  num_class)

print('')
print('y_train.shape = (%d, %d)' % y_train.shape)
print('y_test.shape  = (%d, %d)' % y_test.shape)


In [ ]:
# define model create method
def create_model(num_layer):
    # input layer
    input = Input(shape=(28, 28, 1))
    x     = input
    # hidden layer (convolution)
    for layer_i in range(num_layer):
        # conv
        x = Conv2D(filters=(2 * (layer_i + 1)), 
                   kernel_size=(3, 3), 
                   padding='valid')(x)
        # batch norm
        x = BatchNormalization()(x)
        # activation
        x = Activation("relu")(x)
        # max pool
        x = MaxPooling2D(pool_size=(2, 2))(x)
    # FC
    x = Flatten()(x)
    # 出力層
    y = Dense(units=num_class, 
              activation="softmax")(x)
    # モデル
    model = Model(inputs=input, 
                  outputs=y)
    return model


In [ ]:
# create model
model = create_model(2)

# compile model
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])


In [ ]:
# set param for learning
batch_size  = 128
epochs      = 5


In [ ]:
# learn
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(X_test, y_test))


In [ ]:
score = model.evaluate(x=X_test, 
                       y=y_test, 
                       verbose=0)
print('Test loss     = %.3f' % score[0])
print('Test accuracy = %.3f' % score[1])
